In [44]:
from huggingface_hub import login

login("token")

In [2]:
%%capture
!pip uninstall -y unsloth unsloth-zoo xformers triton bitsandbytes

!pip install --no-cache-dir --upgrade --no-deps \
  git+https://github.com/unslothai/unsloth.git@nightly \
  git+https://github.com/unslothai/unsloth-zoo.git

!pip install xformers==0.0.29.post3
!pip install triton==2.1.0
!pip install bitsandbytes


In [3]:
!pip uninstall -q -y unsloth unsloth-zoo
!pip install -q unsloth


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.8/46.8 kB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 265.6/265.6 kB 21.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.4/10.4 MB 74.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.5/491.5 kB 33.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 489.3/489.3 kB 34.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 162.1/162.1 kB 13.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 318.9/318.9 kB 25.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 138.1/138.1 kB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 74.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 59.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 125.3/125.3 kB 13.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 12.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [21]:
from unsloth import FastLanguageModel
import torch
max_seq_length = 2048
dtype = None
load_in_4bit = True

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/Llama-3.2-3B-Instruct",
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
)

==((====))==  Unsloth 2025.5.6: Fast Llama patching. Transformers: 4.51.3.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.6.0+cu124. CUDA: 7.5. CUDA Toolkit: 12.4. Triton: 3.2.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.29.post3. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


In [22]:
model = FastLanguageModel.get_peft_model(
    model,
    r = 16, # Choose any number > 0 ! Suggested 8, 16, 32, 64, 128
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 16,
    lora_dropout = 0, # Supports any, but = 0 is optimized
    bias = "none",    # Supports any, but = "none" is optimized
    # [NEW] "unsloth" uses 30% less VRAM, fits 2x larger batch sizes!
    use_gradient_checkpointing = "unsloth", # True or "unsloth" for very long context
    random_state = 3407,
    use_rslora = False,  # We support rank stabilized LoRA
    loftq_config = None, # And LoftQ
)

In [45]:
from datasets import load_dataset
dataset = load_dataset("malhajar/alpaca-gpt4-tr")

In [24]:
dataset

DatasetDict({
    train: Dataset({
        features: ['instruction', 'input', 'output', 'text', 'instruction-turkish', 'input-turkish', 'output-turkish', 'text-turkish', 'translation_1', 'translation_2', 'translation_3', 'translation_4', 'number_votes', 'chosen_translation'],
        num_rows: 52002
    })
})

In [25]:
train_data_tr = []
for item in dataset["train"]:
    chunk = {"conversations":[
        {"from":"human",
         "value":item["instruction-turkish"]},
        {"from":"gpt",
         "value":item["output-turkish"]}
    ]}
    train_data_tr.append(chunk)

In [26]:
from datasets import Dataset
dataset_tr = Dataset.from_list(train_data_tr)

In [27]:
dataset_tr

Dataset({
    features: ['conversations'],
    num_rows: 52002
})

In [28]:
from unsloth.chat_templates import get_chat_template

tokenizer = get_chat_template(
    tokenizer,
    chat_template = "llama-3.1",
)

def formatting_prompts_func(examples):
    convos = examples["conversations"]
    texts = [tokenizer.apply_chat_template(convo, tokenize = False, add_generation_prompt = False) for convo in convos]
    return { "text" : texts, }
pass

In [29]:
from unsloth.chat_templates import standardize_sharegpt
dataset = standardize_sharegpt(dataset_tr)
dataset = dataset.map(formatting_prompts_func, batched = True,)

Unsloth: Standardizing formats (num_proc=2):   0%|          | 0/52002 [00:00<?, ? examples/s]

Map:   0%|          | 0/52002 [00:00<?, ? examples/s]

In [30]:
dataset[5]["conversations"]

[{'content': 'Garip olanı tanımlayın.', 'role': 'user'},
 {'content': 'Tuhaf olan ise Telegram. Twitter ve Instagram esas olarak bilgi, resim ve video paylaşımına yönelik sosyal medya platformlarıdır; Telegram ise bulut tabanlı bir anlık mesajlaşma ve IP üzerinden ses hizmetidir.',
  'role': 'assistant'}]

In [31]:
dataset[5]["text"]

'<|begin_of_text|><|start_header_id|>system<|end_header_id|>\n\nCutting Knowledge Date: December 2023\nToday Date: 26 July 2024\n\n<|eot_id|><|start_header_id|>user<|end_header_id|>\n\nGarip olanı tanımlayın.<|eot_id|><|start_header_id|>assistant<|end_header_id|>\n\nTuhaf olan ise Telegram. Twitter ve Instagram esas olarak bilgi, resim ve video paylaşımına yönelik sosyal medya platformlarıdır; Telegram ise bulut tabanlı bir anlık mesajlaşma ve IP üzerinden ses hizmetidir.<|eot_id|>'

In [32]:
from trl import SFTTrainer
from transformers import TrainingArguments, DataCollatorForSeq2Seq
from unsloth import is_bfloat16_supported

trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = dataset,
    dataset_text_field = "text",
    max_seq_length = max_seq_length,
    data_collator = DataCollatorForSeq2Seq(tokenizer = tokenizer),
    dataset_num_proc = 2,
    packing = False, # Can make training 5x faster for short sequences.
    args = TrainingArguments(
        per_device_train_batch_size = 2,
        gradient_accumulation_steps = 4,
        warmup_steps = 5,
        num_train_epochs = 1,
        learning_rate = 2e-4,
        fp16 = not is_bfloat16_supported(),
        bf16 = is_bfloat16_supported(),
        logging_steps = 5,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 3407,
        output_dir = "outputs",
        report_to = "none", # Use this for WandB etc
    ),
)

Unsloth: Tokenizing ["text"] (num_proc=2):   0%|          | 0/52002 [00:00<?, ? examples/s]

In [33]:
from unsloth.chat_templates import train_on_responses_only
trainer = train_on_responses_only(
    trainer,
    instruction_part = "<|start_header_id|>user<|end_header_id|>\n\n",
    response_part = "<|start_header_id|>assistant<|end_header_id|>\n\n",
)

Map (num_proc=2):   0%|          | 0/52002 [00:00<?, ? examples/s]

In [34]:
tokenizer.decode(trainer.train_dataset[5]["input_ids"])

'<|begin_of_text|><|begin_of_text|><|start_header_id|>system<|end_header_id|>\n\nCutting Knowledge Date: December 2023\nToday Date: 26 July 2024\n\n<|eot_id|><|start_header_id|>user<|end_header_id|>\n\nGarip olanı tanımlayın.<|eot_id|><|start_header_id|>assistant<|end_header_id|>\n\nTuhaf olan ise Telegram. Twitter ve Instagram esas olarak bilgi, resim ve video paylaşımına yönelik sosyal medya platformlarıdır; Telegram ise bulut tabanlı bir anlık mesajlaşma ve IP üzerinden ses hizmetidir.<|eot_id|>'

In [35]:
space = tokenizer(" ", add_special_tokens = False).input_ids[0]
tokenizer.decode([space if x == -100 else x for x in trainer.train_dataset[5]["labels"]])

'                                            Tuhaf olan ise Telegram. Twitter ve Instagram esas olarak bilgi, resim ve video paylaşımına yönelik sosyal medya platformlarıdır; Telegram ise bulut tabanlı bir anlık mesajlaşma ve IP üzerinden ses hizmetidir.<|eot_id|>'

In [36]:
#@title Show current memory stats
gpu_stats = torch.cuda.get_device_properties(0)
start_gpu_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
max_memory = round(gpu_stats.total_memory / 1024 / 1024 / 1024, 3)
print(f"GPU = {gpu_stats.name}. Max memory = {max_memory} GB.")
print(f"{start_gpu_memory} GB of memory reserved.")

GPU = Tesla T4. Max memory = 14.741 GB.
6.879 GB of memory reserved.


In [37]:
import os
os.environ["TRITON_DEBUG"] = "1"
os.environ["TRITON_DISABLE_LINE_INFO"] = "1"

trainer_stats = trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 52,002 | Num Epochs = 1 | Total steps = 6,500
O^O/ \_/ \    Batch size per device = 2 | Gradient accumulation steps = 4
\        /    Data Parallel GPUs = 1 | Total batch size (2 x 4 x 1) = 8
 "-____-"     Trainable parameters = 24,313,856/3,000,000,000 (0.81% trained)


Step,Training Loss
5,2.505100
10,2.425800
15,2.577800
20,2.372600
25,2.251400
30,2.307000
35,2.260100
40,2.137300
45,2.297200
50,2.217600


KeyboardInterrupt: 

In [39]:
from unsloth.chat_templates import get_chat_template

tokenizer = get_chat_template(
    tokenizer,
    chat_template = "llama-3.1",
)
FastLanguageModel.for_inference(model) # Enable native 2x faster inference

messages = [
    {"role": "user", "content": "merhaba"},
]
inputs = tokenizer.apply_chat_template(
    messages,
    tokenize = True,
    add_generation_prompt = True, # Must add for generation
    return_tensors = "pt",
).to("cuda")

outputs = model.generate(input_ids = inputs, max_new_tokens = 64, use_cache = True,
                         temperature = 1.5, min_p = 0.1)
tokenizer.batch_decode(outputs)

['<|begin_of_text|><|start_header_id|>system<|end_header_id|>\n\nCutting Knowledge Date: December 2023\nToday Date: 26 July 2024\n\n<|eot_id|><|start_header_id|>user<|end_header_id|>\n\nmerhaba<|eot_id|><|start_header_id|>assistant<|end_header_id|>\n\nMerhaba! Bu seninle iletişime geçmek için bir platform. Ne kadar yardımcı olabilirim?<|eot_id|>']

In [41]:
FastLanguageModel.for_inference(model) # Enable native 2x faster inference

messages = [
    {"role": "user", "content": "merhaba , bana yardımcı olurmusun?"},
]
inputs = tokenizer.apply_chat_template(
    messages,
    tokenize = True,
    add_generation_prompt = True, # Must add for generation
    return_tensors = "pt",
).to("cuda")

from transformers import TextStreamer
text_streamer = TextStreamer(tokenizer, skip_prompt = True)
_ = model.generate(input_ids = inputs, streamer = text_streamer, max_new_tokens = 128,
                   use_cache = True, temperature = 1.5, min_p = 0.1)

Elbette, bana yardımcı olabilirim!<|eot_id|>


In [42]:
model.save_pretrained("lora_model") # Local saving
tokenizer.save_pretrained("lora_model")

('lora_model/tokenizer_config.json',
 'lora_model/special_tokens_map.json',
 'lora_model/tokenizer.json')

In [48]:
model.push_to_hub("canbingol/Llama-3.2-3B-Instruct-tr", token = "token" )
tokenizer.push_to_hub("canbingol/Llama-3.2-3B-Instruct-tr", token = "token")

No files have been modified since last commit. Skipping to prevent empty commit.


Saved model to https://huggingface.co/canbingol/Llama-3.2-3B-Instruct-tr


No files have been modified since last commit. Skipping to prevent empty commit.
